<a href="https://colab.research.google.com/github/paritachavda/Face-Recognition-using-Sparse-Classifier/blob/main/Coil_20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Sparse representation based classifier on Coil-20 dataset**
*  Dataset Info: It has 20 different objects and each object
has 72 different images(Total Images: 1440)
*  Training:30 images per person(600 images)
*  Testing: 42 images per person(840 images)
*  Original dimension: 16384
*  Reduced dimension: 140
*   Accuracy: 95.71




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip -q '/content/drive/My Drive/Datasets/coil-20-proc.zip' 

In [ ]:
import numpy as np
from PIL import Image
import os, sys

In [ ]:
import glob,string
import pandas as pd

path = './coil-20-proc/*.png'
files=glob.glob(path)

In [ ]:
import codecs
from tqdm import tqdm
def contructDataframe(file_list):    
    data=[]
    for file in tqdm(file_list):
        data.append((file,file.split("/")[-1].split("__")[0]))
    return pd.DataFrame(data,columns=['path','label'])

In [ ]:
df=contructDataframe(files)
df

100%|██████████| 1440/1440 [00:00<00:00, 805736.09it/s]


,path,label
0,./coil-20-proc/obj7__46.png,obj7
1,./coil-20-proc/obj9__25.png,obj9
2,./coil-20-proc/obj15__44.png,obj15
3,./coil-20-proc/obj7__58.png,obj7
4,./coil-20-proc/obj5__52.png,obj5
...,...,...
1435,./coil-20-proc/obj13__44.png,obj13
1436,./coil-20-proc/obj4__15.png,obj4
1437,./coil-20-proc/obj10__51.png,obj10
1438,./coil-20-proc/obj18__20.png,obj18


In [ ]:
X,y = [], []
for row in df['path']:
  im = Image.open(row)   
  X.append(np.asarray(im, dtype=np.uint8))

for row in df['label']:
  y.append(int(row[3:]))

In [ ]:
len(X[0][0])

128

In [ ]:
classLabels = np.unique(y)
classLabels

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20])

In [ ]:
partition = [np.where(y==label) for label in classLabels]
partition[0]

(array([  40,   45,   54,   83,  169,  183,  190,  193,  223,  228,  244,
         256,  280,  297,  306,  328,  340,  353,  364,  443,  454,  462,
         469,  526,  529,  547,  552,  559,  568,  572,  585,  587,  628,
         676,  679,  697,  755,  773,  792,  805,  806,  837,  863,  894,
         928,  932,  934,  959,  984, 1063, 1092, 1115, 1126, 1146, 1148,
        1167, 1168, 1170, 1177, 1178, 1181, 1236, 1238, 1239, 1252, 1264,
        1283, 1311, 1351, 1383, 1399, 1430]),)

In [ ]:
X_new = []
y_new = []
number_of_classes = 20

for i in range(number_of_classes):
  for j in partition[i][0]:
    X_new.append(X[j])
    y_new.append(y[j])

In [ ]:
import collections
my_dict = collections.defaultdict(int)
for i in y_new:
    my_dict[i] += 1
temp = min(my_dict.items(), key=lambda x: x[1])
print(temp)
min_num_img = temp[1]
# my_dict

(1, 72)


In [ ]:
np.random.seed(2020)

Xtrain, Xtest = [], []
ytrain, ytest = [], []

img_in_class = 30

for i in range(len(y_new)):
    if i%min_num_img==0:
        train = np.random.choice(min_num_img,img_in_class,replace=False)
        # print(len(train))  
    if ((i%min_num_img in train)):
        Xtrain.append(X_new[i])
        ytrain.append(y_new[i])        
    else:
        Xtest.append(X_new[i])
        ytest.append(y_new[i])      

len(Xtrain)

600

In [ ]:
train

array([42, 40, 58,  1, 28,  2, 16, 30, 46, 29, 49, 55, 59, 48, 65, 64, 69,
       52, 35, 32, 18, 61, 66, 54, 43, 19, 14, 37, 41, 57])

In [ ]:
width =128
height = 128

training_data= np.zeros((number_of_classes*img_in_class,width*height))
training_label= np.zeros(number_of_classes*img_in_class)

for i in range(number_of_classes):
    for c in range(img_in_class):
        training_data[i*img_in_class+c,:] = Xtrain[i*img_in_class+c].flatten()
        training_label[i*img_in_class+c]=ytrain[i*img_in_class+c]    
    
training_data_col = training_data.T


In [ ]:
X_toconcat_test = [np.reshape(e,(Xtest[0].shape[0]*Xtest[0].shape[1],1)) for e in Xtest]
test_data_col = np.concatenate(X_toconcat_test,axis=1) 

In [ ]:
test_data_col.shape

(16384, 840)

In [ ]:
from sklearn.decomposition import PCA

def eigenfaces(X_train,X_test,n_components=200):  
    pca = PCA(n_components=n_components, svd_solver='randomized',
      whiten=True).fit(X_train)
    X_train_pca = pca.transform(X_train)
    X_test_pca = pca.transform(X_test)
    return X_train_pca, X_test_pca  

SRC


In [ ]:
from tqdm import tqdm
from sklearn.linear_model import Lasso

def test(training_label):
    training_label=training_label.astype(int)
    correct_prediction = 0
    
    
    for i in tqdm(range(len(Xtest))):    
        clf = Lasso(alpha=0.01)
        y = Xtest_col[:,i]
        clf.fit(Xtrain_col,y)
        sp_re_test = clf.coef_
        label = ytest[i]
        pred_class_test = np.argmin(residu(y,Xtrain_col,sp_re_test,training_label))
        if(label==pred_class_test):
            correct_prediction=correct_prediction+1

    print('Correct Prediction:',correct_prediction)
    print('Accuracy:',correct_prediction/len(Xtest)*100)


def delta(x,i,classs):
    
    n,m = len(x),len(classs)
    
    if (n != m):
        print('vectors of differents sizes, cannot operate delta')
        
    tmp = i*np.ones(n)-classs

    for k in range(n):
        if tmp[k]==0:
            tmp[k]=1
        else:
            tmp[k]=0 
            
    return tmp*x

def residu(y,A,x,class_x):
    
    k = np.max(class_x)+1
    r = np.zeros(k)
    
    for i in range(0,k):
        r[i] = np.linalg.norm(y - np.dot(A,delta(x,i,class_x)))
        
    return r

In [ ]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
Xtrain_std = ss.fit_transform(training_data_col)
Xtest_std = ss.fit_transform(test_data_col)

Xtrain_PCA, Xtest_PCA = eigenfaces(Xtrain_std.T,Xtest_std.T,n_components=200)
Xtrain_col, Xtest_col = Xtrain_PCA.T, Xtest_PCA.T

In [ ]:
Xtrain_col

array([[ 0.34776033,  0.51807   , -0.65846378, ...,  1.15949244,
         1.03127971,  1.0365162 ],
       [ 1.73691642,  1.28503538,  1.51259991, ..., -1.41805582,
        -1.06798284, -1.07673101],
       [ 0.0258731 , -0.00895447,  0.48444516, ..., -0.19158857,
        -0.33869106, -0.18358351],
       ...,
       [ 0.29081598,  0.4271726 , -0.24031135, ..., -1.36651001,
         1.29661287, -0.96626368],
       [-1.10798743, -0.7231712 ,  0.24232587, ...,  1.47127151,
         0.14315535,  1.73749579],
       [ 0.27615653,  0.73955524, -0.39200352, ...,  2.05001964,
        -0.04757767, -0.18248526]])

In [ ]:
np.random.seed(2020)
from numpy.random import rand

alpha = 0.010
pca_components = 200
number_chosen_components = 140


p_zero = np.random.rand(pca_components,number_chosen_components)
y_tr = p_zero.transpose().dot(Xtrain_col)

In [ ]:
print(p_zero)

[[0.98627683 0.87339195 0.50974552 ... 0.81490306 0.22237219 0.09742508]
 [0.94554092 0.76213503 0.32867765 ... 0.59632419 0.02381255 0.60068745]
 [0.72762319 0.29652697 0.35077751 ... 0.06699604 0.16561218 0.08040271]
 ...
 [0.48614457 0.0283027  0.8678654  ... 0.38063918 0.07358041 0.90425185]
 [0.02619857 0.47531397 0.0345903  ... 0.51420048 0.54840106 0.27919557]
 [0.76017911 0.35783842 0.64871152 ... 0.35476685 0.11377084 0.11586045]]


In [ ]:
y_tr

array([[ 3.34501188,  6.00832733,  4.09065348, ..., -9.67857004,
         6.29342246, -6.66843223],
       [ 1.68278963,  0.52786422,  3.78225734, ..., -0.67304487,
         1.43746985,  2.34630705],
       [ 0.81125609,  1.28939318, -2.53206717, ..., -4.5345603 ,
        -1.3461734 ,  1.78959119],
       ...,
       [10.02814951,  4.3400337 , -0.24657406, ..., -5.95962986,
         0.22474131,  1.6929934 ],
       [10.92473205,  7.61053229, -2.44460609, ..., -7.14318673,
        -1.42630798, -3.17525497],
       [ 3.95714492,  1.77031832,  0.79856697, ...,  3.25609149,
         4.2219974 , -2.94580747]])

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import Lasso
from tqdm import tqdm

sp_re = np.zeros(shape=(y_tr.shape[1]-1,y_tr.shape[1]))
for i in tqdm(range(y_tr.shape[1])):  
    clf = Lasso(alpha)
    y = y_tr[:,i]
    y_tr = np.delete(y_tr,i,1)
    clf.fit(y_tr,y)
    y_tr=np.insert(y_tr,i,y,axis=1)
    sp_re[:,i] = clf.coef_

100%|██████████| 600/600 [01:07<00:00,  8.93it/s]


In [ ]:
sp_re

array([[ 0.        ,  0.02789417,  0.        , ..., -0.        ,
         0.        ,  0.        ],
       [ 0.        , -0.        , -0.        , ..., -0.        ,
         0.        ,  0.        ],
       [ 0.0049398 , -0.        , -0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.0060222 , -0.02211002,  0.        , ..., -0.07618751,
         0.59148122,  0.01730946],
       [-0.        ,  0.        , -0.        , ..., -0.03223609,
        -0.00632556, -0.06172838],
       [ 0.        , -0.        ,  0.        , ..., -0.        ,
         0.01384165,  0.01194502]])

In [ ]:
y_tr.shape

(140, 600)

In [ ]:
mew = np.zeros((number_of_classes, pca_components))
M = np.zeros((1,pca_components))

for i in range(number_of_classes):
    xa = Xtrain_PCA[img_in_class*i:img_in_class*i+img_in_class,:]
    for j in xa:
        mew[i,:] = np.add(mew[i,:],j)
    mew[i,:] = np.divide(mew[i,:],float(len(xa)))

for i in Xtrain_PCA:
    M = np.add(M,i)
M = np.divide(M,float(Xtrain_PCA.shape[0]))

In [ ]:
normalised_wc_proj_sig = np.ndarray(shape=(number_of_classes*img_in_class, pca_components), dtype=np.float64)

for i in range(number_of_classes):
    for j in range(img_in_class):
        normalised_wc_proj_sig[i*img_in_class+j,:] = np.subtract(Xtrain_PCA[i*img_in_class+j,:],mew[i,:])


sw = np.zeros((pca_components,pca_components))

for i in range(number_of_classes):
    xa = normalised_wc_proj_sig[img_in_class*i:img_in_class*i+img_in_class,:]
    xa = xa.transpose()
    cov = np.dot(xa,xa.T)
    sw = sw + cov

normalised_class = np.ndarray(shape=(number_of_classes, pca_components), dtype=np.float64)
sb = np.zeros((pca_components,pca_components))

for i in range(number_of_classes):
    normalised_class[i,:] = np.subtract(mew[i,:],M)

sb = np.dot(normalised_class.T,normalised_class)
sb = np.multiply(sb,float(img_in_class))

In [ ]:
sw

array([[ 8.04852934e+01, -3.81738154e+01, -2.09536210e+01, ...,
        -8.97174482e+00, -3.20863920e+00, -1.08286172e+01],
       [-3.81738154e+01,  8.09123785e+01,  3.75236740e+01, ...,
        -5.21476359e+00, -6.46602205e+00,  1.31742037e+01],
       [-2.09536210e+01,  3.75236740e+01,  1.85324727e+02, ...,
        -2.13966296e+00, -9.56949149e+00,  1.24038913e+01],
       ...,
       [-8.97174482e+00, -5.21476359e+00, -2.13966296e+00, ...,
         5.93940054e+02, -1.44891237e+00, -9.05118275e-02],
       [-3.20863920e+00, -6.46602205e+00, -9.56949149e+00, ...,
        -1.44891237e+00,  5.93522145e+02,  4.92946197e+00],
       [-1.08286172e+01,  1.31742037e+01,  1.24038913e+01, ...,
        -9.05118275e-02,  4.92946197e+00,  5.87509862e+02]])

In [ ]:
sb

array([[ 5.18514707e+02,  3.81738154e+01,  2.09536210e+01, ...,
         8.97174482e+00,  3.20863920e+00,  1.08286172e+01],
       [ 3.81738154e+01,  5.18087622e+02, -3.75236740e+01, ...,
         5.21476359e+00,  6.46602205e+00, -1.31742037e+01],
       [ 2.09536210e+01, -3.75236740e+01,  4.13675273e+02, ...,
         2.13966296e+00,  9.56949149e+00, -1.24038913e+01],
       ...,
       [ 8.97174482e+00,  5.21476359e+00,  2.13966296e+00, ...,
         5.05994620e+00,  1.44891237e+00,  9.05118275e-02],
       [ 3.20863920e+00,  6.46602205e+00,  9.56949149e+00, ...,
         1.44891237e+00,  5.47785469e+00, -4.92946197e+00],
       [ 1.08286172e+01, -1.31742037e+01, -1.24038913e+01, ...,
         9.05118275e-02, -4.92946197e+00,  1.14901380e+01]])

In [ ]:
def delta(x,i,classs):
    
    n,m = len(x),len(classs)
    
    if (n != m):
        print('vectors of differents sizes, cannot operate delta')
        
    tmp = i*np.ones(n)-classs

    for k in range(n):
        if tmp[k]==0:
            tmp[k]=1
        else:
            tmp[k]=0 
            
    return tmp*x

def residu(y,A,x,class_x):
    
    k = np.max(class_x)+1
    r = np.zeros(k)
    
    for i in range(0,k):
        r[i] = np.linalg.norm(y - np.dot(A,delta(x,i,class_x)))
        
    return r

def cost(proj):
    multi_b = np.dot(proj.T,np.dot(sb+Eb,proj))
    numerator = np.trace(multi_b)
    # print(numerator)
    multi_w = np.dot(proj.T,np.dot(Ew+sw,proj))
    denomenator = np.trace(multi_w)
    # print(denomenator)
    final_val = numerator/denomenator
    # print(final_val)
    return final_val

In [ ]:
num_tr = y_tr.shape[1]
Ew = np.zeros((pca_components,pca_components))
Withinclass_error = np.ndarray(shape=(number_of_classes*img_in_class, pca_components), dtype=np.float64)
for i in range(num_tr):
    x = Xtrain_col[:,i]
    label = training_label[i]
    Xtrain_col = np.delete(Xtrain_col,i,1)
    training_label = np.delete(training_label,i)
    Withinclass_error[i,:] = x-np.dot(Xtrain_col,delta(sp_re[:,i],label,training_label))
    Xtrain_col=np.insert(Xtrain_col,i,x,axis=1)
    training_label=np.insert(training_label,i,label)
    
Ew = np.dot(Withinclass_error.T,Withinclass_error)
Ew = Ew/number_of_classes*img_in_class

In [ ]:
Ew

array([[ 1.63420301e+02,  1.74856038e+01, -3.30999445e+01, ...,
        -1.88762248e+01,  1.25527297e+01, -1.00668526e+01],
       [ 1.74856038e+01,  8.93740178e+01,  3.20132781e+01, ...,
         5.50547547e-02, -1.15979318e+01,  9.83715187e+00],
       [-3.30999445e+01,  3.20132781e+01,  9.58647701e+01, ...,
         9.41149886e-01, -2.89625308e+01,  2.07315936e+01],
       ...,
       [-1.88762248e+01,  5.50547547e-02,  9.41149886e-01, ...,
         9.42307438e+02, -6.99231706e+01,  8.06125652e+01],
       [ 1.25527297e+01, -1.15979318e+01, -2.89625308e+01, ...,
        -6.99231706e+01,  1.00137956e+03,  2.47187772e+01],
       [-1.00668526e+01,  9.83715187e+00,  2.07315936e+01, ...,
         8.06125652e+01,  2.47187772e+01,  7.91469203e+02]])

In [ ]:
from tqdm import tqdm
num_data = number_of_classes*img_in_class

classLabels = np.unique(ytrain)
k={}
for label in classLabels:  
    temp = np.where(training_label!=label)
    k.update({label:temp})


In [ ]:
Eb = np.zeros((pca_components,pca_components))
Betweenclass_error = np.ndarray(shape=(number_of_classes*img_in_class, pca_components), dtype=np.float64)
for i in tqdm(range(num_tr)):
    tmp = np.zeros((pca_components))
    x = Xtrain_col[:,i]
    Xtrain_col = np.delete(Xtrain_col,i,1)
    label = training_label[i]
    other_label_index=(k[label][0])
    for j in range(other_label_index.shape[0]):
        other_label = training_label[j]
        training_label = np.delete(training_label,j)
        tmp += x-(np.dot(Xtrain_col,delta(sp_re[:,j],other_label,training_label)))
        training_label=np.insert(training_label,j,other_label)
    Betweenclass_error[i,:] = tmp
    Xtrain_col=np.insert(Xtrain_col,i,x,axis=1)
Eb = np.dot(Betweenclass_error.T,Betweenclass_error)
Eb = Eb/(num_data*(number_of_classes-1))

100%|██████████| 600/600 [04:42<00:00,  2.12it/s]


In [ ]:
def check_symmetric(a, rtol=1e-05, atol=1e-08):
    return np.allclose(a, a.T, rtol=rtol, atol=atol)

In [ ]:
def test(proj_mtx,training_label):
    training_label=training_label.astype(int)
    y_tr = proj_mtx.transpose().dot(Xtrain_col)
    sp_re_test = np.zeros((num_tr,len(Xtest)))
    y_te = proj_mtx.transpose().dot(Xtest_col)

    # print('Finding Sparse repre.:....')
    for i in tqdm(range(y_te.shape[1])):    
        clf = Lasso(alpha=0.01)
        y = y_te[:,i]
        clf.fit(y_tr,y)
        sp_re_test[:,i] = clf.coef_


    pred_class_test = np.zeros(y_te.shape[1])
    for i in tqdm(range(y_te.shape[1])):
        y = y_te[:,i]
        label = ytest[i]
        pred_class_test[i] = np.argmin(residu(y,y_tr,sp_re_test[:,i],training_label))

    pred_class_test = pred_class_test.astype(int)
    correct_prediction = 0
    for i in range(y_te.shape[1]):
        if(ytest[i]==pred_class_test[i]):
            correct_prediction=correct_prediction+1

    print('Correct Prediction:',correct_prediction)
    print('Accuracy:',correct_prediction/y_te.shape[1]*100)


# ***SRC-DP***

In [ ]:
J_dp = np.dot(np.linalg.inv((Ew)),((Eb)))
eigenvalues_dp, eigenvectors_dp, = np.linalg.eig(J_dp)
eig_pairs_dp = [(eigenvalues_dp[index], eigenvectors_dp[:,index]) for index in range(len(eigenvalues_dp))]
eig_pairs_dp.sort(key=lambda tup:tup[0],reverse=True)
eigvalues_sort_dp  = [eig_pairs_dp[index][0] for index in range(len(eigenvalues_dp))]
eigvectors_sort_dp = [eig_pairs_dp[index][1] for index in range(len(eigenvalues_dp))]
print('Number of eigen vectors:',len(eigvalues_sort_dp))
print("k:",number_chosen_components)
proj_mtx_dp = np.array(eigvectors_sort_dp[:number_chosen_components]).transpose()
correct_prediction_dp = test(proj_mtx_dp,training_label)

Number of eigen vectors: 200
k: 140


100%|██████████| 840/840 [00:14<00:00, 57.17it/s]

Correct Prediction: 798
Accuracy: 95.0


# ***SRC-FDC***

In [ ]:
beta = 1
J = np.dot(np.linalg.inv(sw+(beta*Ew)),(sb+(beta*Eb)))
eigenvalues, eigenvectors, = np.linalg.eig(J)
eig_pairs = [(eigenvalues[index], eigenvectors[:,index]) for index in range(len(eigenvalues))]

eig_pairs.sort(key=lambda tup:tup[0],reverse=True)
eigvalues_sort  = [eig_pairs[index][0] for index in range(len(eigenvalues))]
eigvectors_sort = [eig_pairs[index][1] for index in range(len(eigenvalues))]
print('Number of eigen vectors:',len(eigvalues_sort))
print("k:",number_chosen_components)
proj_mtx = np.array(eigvectors_sort[:number_chosen_components]).transpose()
test(proj_mtx,training_label)

Number of eigen vectors: 200
k: 140


100%|██████████| 840/840 [00:13<00:00, 60.67it/s]

Correct Prediction: 795
Accuracy: 94.64285714285714


# ***Average***

In [ ]:
# proj_mtx = np.array(eigvectors_sort[:number_chosen_components]).transpose()
p1 = np.zeros((pca_components,int(number_chosen_components/2)))
index = 0
end = int(number_chosen_components/2) 
for i in range(0,end):
    p1[:,index] = proj_mtx[:,i]+proj_mtx[:,end+i]/2
    # print(i)
    index+=1
test(p1,training_label)

100%|██████████| 840/840 [00:15<00:00, 52.88it/s]

Correct Prediction: 787
Accuracy: 93.69047619047619


# ***Initialization***

In [ ]:
sp_re = np.zeros(shape=(Xtrain_col.shape[1]-1,Xtrain_col.shape[1]))
for i in range(Xtrain_col.shape[1]):  
    clf = Lasso(alpha)
    y = Xtrain_col[:,i]
    Xtrain_col = np.delete(Xtrain_col,i,1)
    clf.fit(Xtrain_col,y)
    Xtrain_col=np.insert(Xtrain_col,i,y,axis=1)
    sp_re[:,i] = clf.coef_

num_tr = Xtrain_col.shape[1]

Ew_new = np.zeros((pca_components,pca_components))
Withinclass_error_new = np.ndarray(shape=(number_of_classes*img_in_class, pca_components), dtype=np.float64)
for i in range(num_tr):
    x = Xtrain_col[:,i]
    label = training_label[i]
    Xtrain_col = np.delete(Xtrain_col,i,1)
    training_label = np.delete(training_label,i)
    Withinclass_error_new[i,:] = x-np.dot(Xtrain_col,delta(sp_re[:,i],label,training_label))
    Xtrain_col=np.insert(Xtrain_col,i,x,axis=1)
    training_label=np.insert(training_label,i,label)

Ew_new = np.dot(Withinclass_error_new.T,Withinclass_error_new)
Ew_new = Ew_new/number_of_classes*img_in_class

 
Eb_new = np.zeros((pca_components,pca_components))
Betweenclass_error_new = np.ndarray(shape=(number_of_classes*img_in_class, pca_components), dtype=np.float64)
for i in tqdm(range(num_tr)):
    tmp = np.zeros((pca_components))
    x = Xtrain_col[:,i]
    Xtrain_col = np.delete(Xtrain_col,i,1)
    label = training_label[i]
    other_label_index=(k[label][0])
    for j in range(other_label_index.shape[0]):
        other_label = training_label[j]
        training_label = np.delete(training_label,j)
        tmp += x-(np.dot(Xtrain_col,delta(sp_re[:,j],other_label,training_label)))
        training_label=np.insert(training_label,j,other_label)
    Betweenclass_error_new[i,:] = tmp
    Xtrain_col=np.insert(Xtrain_col,i,x,axis=1)
Eb_new = np.dot(Betweenclass_error_new.T,Betweenclass_error_new)
Eb_new = Eb_new/(num_data*(number_of_classes-1))


100%|██████████| 600/600 [04:41<00:00,  2.13it/s]


In [ ]:
beta = 1
J = np.dot(np.linalg.inv(sw+(beta*Ew_new)),(sb+(beta*Eb_new)))
eigenvalues, eigenvectors, = np.linalg.eig(J)
eig_pairs = [(eigenvalues[index], eigenvectors[:,index]) for index in range(len(eigenvalues))]

eig_pairs.sort(key=lambda tup:tup[0],reverse=True)
eigvalues_sort  = [eig_pairs[index][0] for index in range(len(eigenvalues))]
eigvectors_sort = [eig_pairs[index][1] for index in range(len(eigenvalues))]
p_zero_1 = np.array(eigvectors_sort[:number_chosen_components]).transpose() 

In [ ]:
p_zero_1=p_zero_1.real
y_tr_init = p_zero_1.transpose().dot(Xtrain_col)
sp_re_init = np.zeros(shape=(y_tr.shape[1]-1,y_tr.shape[1]))
for i in range(y_tr.shape[1]):  
    clf = Lasso(alpha)
    y = y_tr_init[:,i]
    y_tr_init = np.delete(y_tr_init,i,1)
    clf.fit(y_tr_init,y)
    y_tr_init=np.insert(y_tr_init,i,y,axis=1)
    sp_re_init[:,i] = clf.coef_


Ew_1 = np.zeros((pca_components,pca_components))
Withinclass_error_1 = np.ndarray(shape=(number_of_classes*img_in_class, pca_components), dtype=np.float64)
for i in range(num_tr):
    x = Xtrain_col[:,i]
    label = training_label[i]
    Xtrain_col = np.delete(Xtrain_col,i,1)
    training_label = np.delete(training_label,i)
    Withinclass_error_1[i,:] = x-np.dot(Xtrain_col,delta(sp_re_init[:,i],label,training_label))
    Xtrain_col=np.insert(Xtrain_col,i,x,axis=1)
    training_label=np.insert(training_label,i,label)

Ew_1 = np.dot(Withinclass_error_1.T,Withinclass_error_1)
Ew_1 = Ew_1/number_of_classes*img_in_class


Eb_1 = np.zeros((pca_components,pca_components))
Betweenclass_error_1 = np.ndarray(shape=(number_of_classes*img_in_class, pca_components), dtype=np.float64)
for i in tqdm(range(num_tr)):
    tmp = np.zeros((pca_components))
    x = Xtrain_col[:,i]
    Xtrain_col = np.delete(Xtrain_col,i,1)
    label = training_label[i]
    other_label_index=(k[label][0])
    for j in range(other_label_index.shape[0]):
        other_label = training_label[j]
        training_label = np.delete(training_label,j)
        tmp += x-(np.dot(Xtrain_col,delta(sp_re_init[:,j],other_label,training_label)))
        training_label=np.insert(training_label,j,other_label)
    Betweenclass_error_1[i,:] = tmp
    Xtrain_col=np.insert(Xtrain_col,i,x,axis=1)
Eb_1 = np.dot(Betweenclass_error_1.T,Betweenclass_error_1)
Eb_1 = Eb_1/(num_data*(number_of_classes-1))

J = np.dot(np.linalg.inv(sw+(beta*Ew_1)),sb+(beta*Eb_1))
eigenvalues, eigenvectors, = np.linalg.eig(J)
eig_pairs = [(eigenvalues[index], eigenvectors[:,index]) for index in range(len(eigenvalues))]

eig_pairs.sort(key=lambda tup:tup[0],reverse=True)
eigvalues_sort  = [eig_pairs[index][0] for index in range(len(eigenvalues))]
eigvectors_sort = [eig_pairs[index][1] for index in range(len(eigenvalues))]
proj_mtx_new = np.array(eigvectors_sort[:number_chosen_components]).transpose()

100%|██████████| 600/600 [04:39<00:00,  2.15it/s]


In [ ]:
proj_mtx_new = proj_mtx_new.real
test(proj_mtx_new,training_label)

100%|██████████| 840/840 [00:13<00:00, 64.10it/s]

Correct Prediction: 804
Accuracy: 95.71428571428572


In [ ]:
p1 = np.zeros((pca_components,int(number_chosen_components/2)))
index = 0
end = int(number_chosen_components/2) 
for i in range(0,end):
    p1[:,index] = proj_mtx_new[:,i]+proj_mtx_new[:,end+i]/2
    # print(i)
    index+=1
test(p1,training_label)

100%|██████████| 840/840 [00:12<00:00, 66.15it/s]

Correct Prediction: 801
Accuracy: 95.35714285714286
